<a href="https://colab.research.google.com/github/elangbijak4/Blockchain_Research/blob/main/Rev7_Framework_Mini1_Infrastruktur_Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Tool Dasar

In [1]:
import hashlib
import time
import json
import random
import os

### Class Block

In [2]:
class Block:
    def __init__(self, index, previous_hash, timestamp, data, hash):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = hash

    @staticmethod
    def calculate_hash(index, previous_hash, timestamp, data):
        value = f"{index}{previous_hash}{timestamp}{data}"
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

    @staticmethod
    def create_genesis_block():
        return Block(0, "0", int(time.time()), "Genesis Block", Block.calculate_hash(0, "0", int(time.time()), "Genesis Block"))

    def to_dict(self):
        return {
            "index": self.index,
            "previous_hash": self.previous_hash,
            "timestamp": self.timestamp,
            "data": self.data,
            "hash": self.hash
        }

### Protokol Proof of Elapsed Time

In [3]:
class ProofOfElapsedTime:
    def __init__(self):
        self.wait_times = {}

    def wait_for_block(self, address):
        wait_time = random.uniform(0, 10)  # Simulating wait time
        self.wait_times[address] = wait_time
        time.sleep(wait_time)

    def select_validator(self):
        return min(self.wait_times, key=self.wait_times.get)

### Protokol Proof of Burn

In [4]:
class ProofOfBurn:
    def __init__(self):
        self.burned_coins = {}  # To keep track of burned coins by address

    def burn_coins(self, address, amount):
        if address in self.burned_coins:
            self.burned_coins[address] += amount
        else:
            self.burned_coins[address] = amount

    def select_validator(self):
        total_burned = sum(self.burned_coins.values())
        pick = random.uniform(0, total_burned)
        current = 0
        for address, burned in self.burned_coins.items():
            current += burned
            if current > pick:
                return address

### Protokol Proof Of Work

In [5]:
class ProofOfWork:
    def __init__(self, difficulty=2):
        self.difficulty = difficulty

    def mine_block(self, block):
        while block.hash[:self.difficulty] != "0" * self.difficulty:
            block.nonce += 1
        return block

    def is_valid_block(self, block):
        return block.hash[:self.difficulty] == "0" * self.difficulty

    def get_reward(self, address):
        return 50  # Simulating reward for mining a block

    def store_smart_contract(self, blockchain, code):
        latest_block = blockchain.get_latest_block()
        index = latest_block.index + 1
        timestamp = int(time.time())
        previous_hash = latest_block.hash
        smart_contract = SmartContract(code)  # Assuming SmartContract is defined
        new_block = Block(index, previous_hash, timestamp, code, "")

        # In Proof of Work, we don't have validators, so we just indicate the system
        print(f"Smart Contract Block #{index} created by the system using PoW")

        new_block.hash = Block.calculate_hash(new_block.index, new_block.previous_hash, new_block.timestamp, new_block.data)
        blockchain.chain.append(new_block)
        block_hash = new_block.hash
        blockchain.smart_contracts[block_hash] = smart_contract
        return block_hash

### Protokol Proof of ByzantineFaultTolerance

In [6]:
class ByzantineFaultTolerance:
    def __init__(self, validators, faulty_validators=None):
        self.validators = validators
        self.faulty_validators = faulty_validators if faulty_validators else []
        self.leader = None

    def select_leader(self):
        non_faulty_validators = [v for v in self.validators if v not in self.faulty_validators]
        self.leader = random.choice(non_faulty_validators)
        return self.leader

    def select_validator(self, chain_length):
        non_faulty_validators = [v for v in self.validators if v not in self.faulty_validators]
        return random.choice(non_faulty_validators)

    def validate_block(self, block, chain):
        previous_block = chain[block.index - 1]
        if block.previous_hash != previous_block.hash:
            return False
        if block.hash != block.calculate_hash():
            return False
        return True

    def add_faulty_validator(self, validator):
        if validator not in self.faulty_validators:
            self.faulty_validators.append(validator)

    def remove_faulty_validator(self, validator):
        if validator in self.faulty_validators:
            self.faulty_validators.remove(validator)

    def reach_consensus(self, block, chain):
        votes = {validator: self.validate_block(block, chain) for validator in self.validators if validator not in self.faulty_validators}
        positive_votes = sum(votes.values())
        if positive_votes > len(self.validators) * 2 / 3:
            return True
        return False

    def calculate_reward(self, validator):
        # For simplicity, we set a fixed reward. In practice, it might depend on various factors.
        return 10

### Protokol Proof of Authority

In [7]:
class ProofOfAuthority:
    def __init__(self, authorities):
        self.authorities = authorities

    def select_validator(self, chain_length):
        return random.choice(self.authorities)

    def validate_block(self, block, chain):
        previous_block = chain[block.index - 1]
        if block.previous_hash != previous_block.hash:
            return False
        if block.hash != block.calculate_hash():
            return False
        return True

    def reach_consensus(self, block, chain):
        return self.validate_block(block, chain)

    def calculate_reward(self, validator):
        # For simplicity, we set a fixed reward. In practice, it might depend on various factors.
        return 10

### Protokol Proof of Stake

In [8]:
class ProofOfStake:
    def __init__(self):
        self.stakeholders = {}  # To keep track of stakeholder's stakes

    def add_stakeholder(self, address, stake):
        if address in self.stakeholders:
            self.stakeholders[address] += stake
        else:
            self.stakeholders[address] = stake

    def select_validator(self):
        total_stake = sum(self.stakeholders.values())
        pick = random.uniform(0, total_stake)
        current = 0
        for address, stake in self.stakeholders.items():
            current += stake
            if current > pick:
                return address

### Protokol Proof Of Participants

In [51]:
class ProofOfParticipants:
    def __init__(self):
        self.participants = {}

    def add_participant(self, address):
        if address not in self.participants:
            self.participants[address] = 0

    def record_participation(self, address):
        if address in self.participants:
            self.participants[address] += 1

    #def select_validator(self, current_block_index):
    #    if not self.participants:
    #        return None
    #    return max(self.participants, key=self.participants.get)

    def select_validator(self, current_block_index):
        if not self.participants:
            return None
        # Simple round-robin selection based on block index
        index = current_block_index % len(self.participants)
        return list(self.participants.keys())[index]


    def reset_participation(self):
        for address in self.participants:
            self.participants[address] = 0

### Class Blockchain

In [52]:
class Blockchain:
    def __init__(self, consensus_protocol, filename='blockchain.json'):
        self.filename = filename
        self.consensus_protocol = consensus_protocol
        self.chain = self.load_chain()

    def load_chain(self):
        if os.path.exists(self.filename):
            with open(self.filename, 'r') as file:
                data = json.load(file)
                return [Block(**block) for block in data]
        else:
            return [Block.create_genesis_block()]

    def save_chain(self):
        with open(self.filename, 'w') as file:
            json.dump([block.to_dict() for block in self.chain], file, indent=4)

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, data):
        latest_block = self.get_latest_block()
        index = latest_block.index + 1
        timestamp = int(time.time())
        previous_hash = latest_block.hash
        new_block = Block(index, previous_hash, timestamp, data, "")

        if isinstance(self.consensus_protocol, ProofOfBurn):
            validator = self.consensus_protocol.select_validator()
            print(f"Block #{index} created by {validator} using PoB")
        if isinstance(self.consensus_protocol, ProofOfStake):
            validator = self.consensus_protocol.select_validator()
            print(f"Block #{index} created by {validator} using PoS")
        if isinstance(self.consensus_protocol, ByzantineFaultTolerance):
            validator = self.consensus_protocol.select_validator(len(self.chain))
            print(f"Block #{index} created by {validator} using BFT")
        if isinstance(self.consensus_protocol, ProofOfAuthority):
            validator = self.consensus_protocol.select_validator(len(self.chain))
            print(f"Block #{index} created by {validator} using PoA")
        if isinstance(self.consensus_protocol, ProofOfParticipants):
            validator = self.consensus_protocol.select_validator(len(self.chain))  # Pass the current block index
            print(f"Block #{index} created by {validator} using PoP")

        elif isinstance(self.consensus_protocol, ProofOfElapsedTime):
            for address in self.consensus_protocol.wait_times.keys():
                self.consensus_protocol.wait_for_block(address)
            validator = self.consensus_protocol.select_validator()
            print(f"Block #{index} created by {validator} using PoET")

        new_block.hash = Block.calculate_hash(new_block.index, new_block.previous_hash, new_block.timestamp, new_block.data)
        self.chain.append(new_block)
        self.save_chain()

    def to_json(self):
        self.save_chain()
        with open(self.filename, 'r') as file:
            return file.read()

### Class Smart Contract

In [11]:
class SmartContract:
    def __init__(self, code):
        self.code = code
        self.storage = {}

    def execute(self, blockchain, **kwargs):
        local_context = {
            'blockchain': blockchain,
            'storage': self.storage
        }
        local_context.update(kwargs)
        exec(self.code, {}, local_context)
        self.storage = local_context['storage']

### Implementasi ERC721 (Interface yang diimplementasikan memimik ERC721)

In [12]:
class ERC721:
    def __init__(self):
        self.tokens = {}
        self.token_owners = {}
        self.token_metadata = {}
        self._token_owner = {}  # tokenId => owner
        self._owned_tokens = {}  # owner => list of tokenIds
        self._token_approvals = {}  # tokenId => approved address
        self._operator_approvals = {}  # owner => (operator => approved)

    def _mint(self, to, tokenId, metadata):
        if to in self._owned_tokens:
            self._owned_tokens[to].append(tokenId)
        else:
            self._owned_tokens[to] = [tokenId]
        self._token_owner[tokenId] = to
        self.token_metadata[tokenId] = metadata

    def _transfer(self, from_, to, tokenId):
        self._owned_tokens[from_].remove(tokenId)
        if to in self._owned_tokens:
            self._owned_tokens[to].append(tokenId)
        else:
            self._owned_tokens[to] = [tokenId]
        self._token_owner[tokenId] = to

    def mint(self, to, metadata):
        tokenId = hashlib.sha256(metadata.encode()).hexdigest()
        self._mint(to, tokenId, metadata)
        return tokenId

    def transfer(self, from_, to, tokenId):
        self._transfer(from_, to, tokenId)

### Implementasi ERC20 (Interface yang diimplementasikan memimik ERC20)

In [13]:
class ERC20:
    def __init__(self):
        self.balances = {}
        self.total_supply = 0

    def mint(self, to, amount):
        if to in self.balances:
            self.balances[to] += amount
        else:
            self.balances[to] = amount
        self.total_supply += amount

    def transfer(self, from_, to, amount):
        if from_ not in self.balances or self.balances[from_] < amount:
            raise ValueError("Insufficient balance")
        if to in self.balances:
            self.balances[to] += amount
        else:
            self.balances[to] = amount
        self.balances[from_] -= amount

    def balance_of(self, address):
        return self.balances.get(address, 0)

### Aplikasi Sistem Blockchain MyBlockchain

In [53]:
class MyBlockchain(Blockchain):
    def __init__(self,pos):
        super().__init__(pos)
        self.smart_contracts = {}
        self.nft_contract = ERC721()
        self.fungible_token_contract = ERC20()
        self.pos = pos

    def store_smart_contract(self, code):
        latest_block = self.get_latest_block()
        index = latest_block.index + 1
        timestamp = int(time.time())
        previous_hash = latest_block.hash
        smart_contract = SmartContract(code)
        new_block = Block(index, previous_hash, timestamp, code, "")
        validator = self.pos.select_validator()
        print(f"Smart Contract Block #{index} created by {validator} using PoS")
        new_block.hash = Block.calculate_hash(new_block.index, new_block.previous_hash, new_block.timestamp, new_block.data)
        self.chain.append(new_block)
        block_hash = new_block.hash
        self.smart_contracts[block_hash] = smart_contract
        return block_hash

    def execute_smart_contract_from_block(self, block_hash, **kwargs):
        if block_hash in self.smart_contracts:
            smart_contract = self.smart_contracts[block_hash]
            smart_contract.execute(self, **kwargs)
        else:
            print("Smart contract not found in blockchain")

    def mint_nft(self, owner, metadata):
        tokenId = self.nft_contract.mint(owner, metadata)
        data = json.dumps({
            "type": "mint",
            "owner": owner,
            "tokenId": tokenId,
            "metadata": metadata
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def transfer_nft(self, from_, to, tokenId):
        self.nft_contract.transfer(from_, to, tokenId)
        data = json.dumps({
            "type": "transfer",
            "from": from_,
            "to": to,
            "tokenId": tokenId
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def mint_fungible_token(self, to, amount):
        self.fungible_token_contract.mint(to, amount)
        data = json.dumps({
            "type": "mint_fungible",
            "to": to,
            "amount": amount
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def transfer_fungible_token(self, from_, to, amount):
        self.fungible_token_contract.transfer(from_, to, amount)
        data = json.dumps({
            "type": "transfer_fungible",
            "from": from_,
            "to": to,
            "amount": amount
        })
        self.add_block(data)
        return self.get_latest_block().hash

### Demo Proof Of Work

In [54]:
# Create a Proof of Work consensus protocol
pow_protocol = ProofOfWork(difficulty=2)

# Create a MyBlockchain instance using the Proof of Work consensus protocol
blockchain = MyBlockchain(pow_protocol)

# Add some blocks to the blockchain
blockchain.add_block("Block 1 data")
blockchain.add_block("Block 2 data")
blockchain.add_block("Block 3 data")

# Print the blockchain
print(blockchain.chain)
print(blockchain.to_json())

[<__main__.Block object at 0x7e4ab97c6ad0>, <__main__.Block object at 0x7e4ab97c5bd0>, <__main__.Block object at 0x7e4ab97c4580>, <__main__.Block object at 0x7e4ab97c62f0>, <__main__.Block object at 0x7e4ab97c6800>, <__main__.Block object at 0x7e4ab97c7760>, <__main__.Block object at 0x7e4ab97c41c0>, <__main__.Block object at 0x7e4ab97c4c10>, <__main__.Block object at 0x7e4ab97c6650>, <__main__.Block object at 0x7e4ab97c74c0>, <__main__.Block object at 0x7e4ab97c7130>, <__main__.Block object at 0x7e4ab97c7910>, <__main__.Block object at 0x7e4ab97c48e0>, <__main__.Block object at 0x7e4ab97c5ff0>, <__main__.Block object at 0x7e4ab97c5ed0>, <__main__.Block object at 0x7e4ab97c4bb0>, <__main__.Block object at 0x7e4ab97c6920>, <__main__.Block object at 0x7e4ab97c5d50>, <__main__.Block object at 0x7e4ab97c41f0>, <__main__.Block object at 0x7e4ab97c4310>, <__main__.Block object at 0x7e4ab97c7df0>, <__main__.Block object at 0x7e4ab97c7a90>, <__main__.Block object at 0x7e4ab97c5c60>, <__main__.

In [55]:
# Create a smart contract
code = """
def transfer(from_, to, amount):
    if from_ not in balances or balances[from_] < amount:
        raise ValueError("Insufficient balance")
    if to in balances:
        balances[to] += amount
    else:
        balances[to] = amount
    balances[from_] -= amount
"""

# Store the smart contract
block_hash = pow_protocol.store_smart_contract(blockchain, code) # Call the method on pow_protocol

Smart Contract Block #24 created by the system using PoW


In [56]:
# Execute the smart contract
blockchain.execute_smart_contract_from_block(block_hash, from_="Alice", to="Bob", amount=100)

In [57]:
# Print the balances
print(blockchain.fungible_token_contract.balances)

{}


In [58]:
print(blockchain.to_json())

[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1721140664,
        "data": "Genesis Block",
        "hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae"
    },
    {
        "index": 1,
        "previous_hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae",
        "timestamp": 1721140664,
        "data": "Block 1 data",
        "hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6"
    },
    {
        "index": 2,
        "previous_hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6",
        "timestamp": 1721140664,
        "data": "Block 2 data",
        "hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15"
    },
    {
        "index": 3,
        "previous_hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15",
        "timestamp": 1721140664,
        "data": "Block 3 data",
        "hash": "d004e80e3d6a433c5c3e66dca160c1fb0f9b6

### Demo Proof of Stake

In [59]:
# Contoh Penggunaan
pos = ProofOfStake()
my_blockchain = MyBlockchain(pos)

In [60]:
# Add stakeholders using the ProofOfStake object
pos.add_stakeholder('Alice', 10)
pos.add_stakeholder('Bob', 20)
pos.add_stakeholder('Charlie', 30)

In [61]:
# Menambahkan blok biasa
my_blockchain.add_block('Block 1 Data')
my_blockchain.add_block('Block 2 Data')

Block #25 created by Charlie using PoS
Block #26 created by Bob using PoS


In [62]:
# Menambahkan smart contract
smart_contract_code = """
def contract_function(storage):
    storage['counter'] = storage.get('counter', 0) + 1
    print(f"Counter value: {storage['counter']}")
contract_function(storage)
"""

In [63]:
block_hash = my_blockchain.store_smart_contract(smart_contract_code)
print(f"Smart contract stored in block with hash: {block_hash}")

Smart Contract Block #27 created by Bob using PoS
Smart contract stored in block with hash: 5800e3fc56b6aa5d75355334348c8b429d693331d0e68d8220acdac9bef00a04


In [64]:
# Menjalankan smart contract dari blockchain
my_blockchain.execute_smart_contract_from_block(block_hash)

Counter value: 1


In [65]:
# Menjalankan kembali smart contract untuk melihat perubahan state
my_blockchain.execute_smart_contract_from_block(block_hash)

Counter value: 2


In [66]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(my_blockchain.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1721140664,
        "data": "Genesis Block",
        "hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae"
    },
    {
        "index": 1,
        "previous_hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae",
        "timestamp": 1721140664,
        "data": "Block 1 data",
        "hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6"
    },
    {
        "index": 2,
        "previous_hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6",
        "timestamp": 1721140664,
        "data": "Block 2 data",
        "hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15"
    },
    {
        "index": 3,
        "previous_hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15",
        "timestamp": 1721140664,
        "data": "Block 3 data",
        "hash": "d004e80e3

In [67]:
# Membuat NFT baru
owner_address = "0xfeedfacefeedfacefeedfacefeedfacefeedface"
asset_uri = "https://example2.co.id"
mint_block_hash = my_blockchain.mint_nft(owner_address, asset_uri)
print(f"NFT minted in block with hash: {mint_block_hash}")

Block #28 created by Bob using PoS
NFT minted in block with hash: c73e57ac86c4b70e0471e8e89a25f1e1db20c35faeb3b103b788f70dd617fd88


In [68]:
# Transfer NFT
new_owner_address = "0xbadc0ffeebadc0ffeebadc0ffeebadc0ffeebadc"
token_id = hashlib.sha256(asset_uri.encode()).hexdigest()
transfer_block_hash = my_blockchain.transfer_nft(owner_address, new_owner_address, token_id)
print(f"NFT transferred in block with hash: {transfer_block_hash}")

Block #29 created by Charlie using PoS
NFT transferred in block with hash: 44cae7c5413af8dda8794172056526cd2ddb0dcc6afb7c08598ec105873c0e85


In [69]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(my_blockchain.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1721140664,
        "data": "Genesis Block",
        "hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae"
    },
    {
        "index": 1,
        "previous_hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae",
        "timestamp": 1721140664,
        "data": "Block 1 data",
        "hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6"
    },
    {
        "index": 2,
        "previous_hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6",
        "timestamp": 1721140664,
        "data": "Block 2 data",
        "hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15"
    },
    {
        "index": 3,
        "previous_hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15",
        "timestamp": 1721140664,
        "data": "Block 3 data",
        "hash": "d004e80e3

In [70]:
# Demonstrasi Token Fungible (ERC20)
my_blockchain.mint_fungible_token("0xabc", 1000)
my_blockchain.mint_fungible_token("0xdef", 500)

Block #30 created by Charlie using PoS
Block #31 created by Bob using PoS


'e40af7a76d7f46fe13048dc3f92264a345fc6cf0b793d64e1dda26fec47a3010'

In [71]:
print("Balance of 0xabc:", my_blockchain.fungible_token_contract.balance_of("0xabc"))
print("Balance of 0xdef:", my_blockchain.fungible_token_contract.balance_of("0xdef"))

Balance of 0xabc: 1000
Balance of 0xdef: 500


In [72]:
my_blockchain.transfer_fungible_token("0xabc", "0xdef", 200)

Block #32 created by Charlie using PoS


'f045331cb60f3ff9e2e6128c208eda1247530fe583d61d343dba79ce368f24e9'

In [73]:
print("Balance of 0xabc after transfer:", my_blockchain.fungible_token_contract.balance_of("0xabc"))
print("Balance of 0xdef after transfer:", my_blockchain.fungible_token_contract.balance_of("0xdef"))

Balance of 0xabc after transfer: 800
Balance of 0xdef after transfer: 700


In [74]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(my_blockchain.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1721140664,
        "data": "Genesis Block",
        "hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae"
    },
    {
        "index": 1,
        "previous_hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae",
        "timestamp": 1721140664,
        "data": "Block 1 data",
        "hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6"
    },
    {
        "index": 2,
        "previous_hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6",
        "timestamp": 1721140664,
        "data": "Block 2 data",
        "hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15"
    },
    {
        "index": 3,
        "previous_hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15",
        "timestamp": 1721140664,
        "data": "Block 3 data",
        "hash": "d004e80e3

### Demo Proof of Burn

In [75]:
pob = ProofOfBurn()

In [76]:
pob.burn_coins('Alice', 10)
pob.burn_coins('Bob', 20)
pob.burn_coins('Charlie', 30)

In [77]:
blockchain_pob = Blockchain(pob)
blockchain_pob.add_block('Block 1 Data')
blockchain_pob.add_block('Block 2 Data')

Block #33 created by Charlie using PoB
Block #34 created by Alice using PoB


In [78]:
print("Blockchain content with PoB in JSON:")
print(blockchain_pob.to_json())

Blockchain content with PoB in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1721140664,
        "data": "Genesis Block",
        "hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae"
    },
    {
        "index": 1,
        "previous_hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae",
        "timestamp": 1721140664,
        "data": "Block 1 data",
        "hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6"
    },
    {
        "index": 2,
        "previous_hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6",
        "timestamp": 1721140664,
        "data": "Block 2 data",
        "hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15"
    },
    {
        "index": 3,
        "previous_hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15",
        "timestamp": 1721140664,
        "data": "Block 3 data",
        "hash": "

### Demo Proof of Elapsed Time

In [79]:
poet = ProofOfElapsedTime()
poet.wait_times = {'Alice': 0, 'Bob': 0, 'Charlie': 0}

In [80]:
blockchain_poet = Blockchain(poet)
blockchain_poet.add_block('Block 1 Data')
blockchain_poet.add_block('Block 2 Data')

Block #35 created by Alice using PoET
Block #36 created by Alice using PoET


In [42]:
print("Blockchain content with PoET in JSON:")
print(blockchain_poet.to_json())

Blockchain content with PoET in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1721140664,
        "data": "Genesis Block",
        "hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae"
    },
    {
        "index": 1,
        "previous_hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae",
        "timestamp": 1721140664,
        "data": "Block 1 data",
        "hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6"
    },
    {
        "index": 2,
        "previous_hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6",
        "timestamp": 1721140664,
        "data": "Block 2 data",
        "hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15"
    },
    {
        "index": 3,
        "previous_hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15",
        "timestamp": 1721140664,
        "data": "Block 3 data",
        "hash": 

### Demo Proof of Byzantine Fault Tolerance

In [81]:
validators = ["Alice", "Bob", "Charlie", "David"]
faulty_validators = ["Charlie"]

bft = ByzantineFaultTolerance(validators, faulty_validators)

In [82]:
blockchain_bft = Blockchain(bft)
blockchain_bft.add_block('Block 1 Data')
blockchain_bft.add_block('Block 2 Data')

Block #37 created by Bob using BFT
Block #38 created by Bob using BFT


In [83]:
print("Blockchain content with BFT in JSON:")
print(blockchain_bft.to_json())

Blockchain content with BFT in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1721140664,
        "data": "Genesis Block",
        "hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae"
    },
    {
        "index": 1,
        "previous_hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae",
        "timestamp": 1721140664,
        "data": "Block 1 data",
        "hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6"
    },
    {
        "index": 2,
        "previous_hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6",
        "timestamp": 1721140664,
        "data": "Block 2 data",
        "hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15"
    },
    {
        "index": 3,
        "previous_hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15",
        "timestamp": 1721140664,
        "data": "Block 3 data",
        "hash": "

### Demo Proof of Authority

In [84]:
authorities = ["Alice", "Bob", "Charlie"]

poa = ProofOfAuthority(authorities)

In [85]:
blockchain_poa = Blockchain(poa)
blockchain_poa.add_block('Block 1 Data')
blockchain_poa.add_block('Block 2 Data')

Block #39 created by Bob using PoA
Block #40 created by Alice using PoA


In [86]:
print("Blockchain content with PoA in JSON:")
print(blockchain_poa.to_json())

Blockchain content with PoA in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1721140664,
        "data": "Genesis Block",
        "hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae"
    },
    {
        "index": 1,
        "previous_hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae",
        "timestamp": 1721140664,
        "data": "Block 1 data",
        "hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6"
    },
    {
        "index": 2,
        "previous_hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6",
        "timestamp": 1721140664,
        "data": "Block 2 data",
        "hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15"
    },
    {
        "index": 3,
        "previous_hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15",
        "timestamp": 1721140664,
        "data": "Block 3 data",
        "hash": "

### Demo Proof Of Participants

In [87]:
# Create a Proof of Participants consensus protocol
pop_protocol = ProofOfParticipants()

# Register participants
pop_protocol.add_participant("Alice")
pop_protocol.add_participant("Bob")
pop_protocol.add_participant("Charlie")

# Create a MyBlockchain instance using the Proof of Participants consensus protocol
blockchain_pop = MyBlockchain(pop_protocol)

# Add some blocks to the blockchain
blockchain_pop.add_block("Block 1 data")
blockchain_pop.add_block("Block 2 data")
blockchain_pop.add_block("Block 3 data")

# Print the blockchain
for block in blockchain_pop.chain:
    print(block.to_dict())

print(blockchain_pop.to_json())

Block #41 created by Charlie using PoP
Block #42 created by Alice using PoP
Block #43 created by Bob using PoP
{'index': 0, 'previous_hash': '0', 'timestamp': 1721140664, 'data': 'Genesis Block', 'hash': '88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae'}
{'index': 1, 'previous_hash': '88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae', 'timestamp': 1721140664, 'data': 'Block 1 data', 'hash': 'dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6'}
{'index': 2, 'previous_hash': 'dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6', 'timestamp': 1721140664, 'data': 'Block 2 data', 'hash': 'e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15'}
{'index': 3, 'previous_hash': 'e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15', 'timestamp': 1721140664, 'data': 'Block 3 data', 'hash': 'd004e80e3d6a433c5c3e66dca160c1fb0f9b6edf086319022807fd17a0bb8fa4'}
{'index': 4, 'previous_hash': 'd004e80e3d6a433c5c3e66dca160

## DAO

### Demo DAO Grant DAOs

In [88]:
class GrantDAOs(MyBlockchain):
    def __init__(self,pos):
        super().__init__(pos)
        self.grants = {}

    def create_grant(self, grant_id, proposer, amount, description):
        self.grants[grant_id] = {
            'proposer': proposer,
            'amount': amount,
            'description': description,
            'votes': {'yes': 0, 'no': 0},
            'executed': False
        }
        data = json.dumps({
            "type": "create_grant",
            "grant_id": grant_id,
            "proposer": proposer,
            "amount": amount,
            "description": description
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def vote_grant(self, grant_id, stakeholder, vote):
        if grant_id in self.grants:
            if vote not in ['yes', 'no']:
                raise ValueError("Vote must be 'yes' atau 'no'")
            self.grants[grant_id]['votes'][vote] += 1
            data = json.dumps({
                "type": "vote_grant",
                "grant_id": grant_id,
                "stakeholder": stakeholder,
                "vote": vote
            })
            self.add_block(data)
            return self.get_latest_block().hash
        else:
            raise ValueError("Grant ID tidak ditemukan")

    def execute_grant(self, grant_id):
        if grant_id in self.grants:
            grant = self.grants[grant_id]
            if grant['votes']['yes'] > grant['votes']['no'] and not grant['executed']:
                self.reward_player(grant['proposer'], grant['amount'])
                grant['executed'] = True
                data = json.dumps({
                    "type": "execute_grant",
                    "grant_id": grant_id
                })
                self.add_block(data)
                return self.get_latest_block().hash
            else:
                raise ValueError("Grant belum disetujui atau sudah dieksekusi")
        else:
            raise ValueError("Grant ID tidak ditemukan")

    def reward_player(self, player, amount):
        self.fungible_token_contract.mint(player, amount)
        data = json.dumps({
            "type": "reward_player",
            "player": player,
            "amount": amount
        })
        self.add_block(data)
        return self.get_latest_block().hash

In [89]:
# Demonstrasi Penggunaan
pos = ProofOfStake()
grant_daos = GrantDAOs(pos)

pos.add_stakeholder('Alice', 10)
pos.add_stakeholder('Bob', 20)
pos.add_stakeholder('Charlie', 30)
pos.add_stakeholder('Herman', 15)
pos.add_stakeholder('Hani', 7)
pos.add_stakeholder('Dolbi', 50)

# Menambahkan blok biasa
grant_daos.add_block('Block 1 Data')
grant_daos.add_block('Block 2 Data')

Block #44 created by Dolbi using PoS
Block #45 created by Dolbi using PoS


In [90]:
# Membuat proposal grant
grant_id = "grant1"
proposer = "Alice"
amount = 100
description = "Research on sustainable blockchain solutions"
create_grant_hash = grant_daos.create_grant(grant_id, proposer, amount, description)
print(f"Grant proposal created in block with hash: {create_grant_hash}")

Block #46 created by Charlie using PoS
Grant proposal created in block with hash: d0c408feba362054cd302c7ab8dc7065417a3fe2fd9252f25099548c4fbb5310


In [91]:
# Stakeholder voting
vote_hash_1 = grant_daos.vote_grant(grant_id, 'Bob', 'yes')
print(f"Grant vote by Bob in block with hash: {vote_hash_1}")

vote_hash_2 = grant_daos.vote_grant(grant_id, 'Charlie', 'no')
print(f"Grant vote by Charlie in block with hash: {vote_hash_2}")

vote_hash_3 = grant_daos.vote_grant(grant_id, 'Hani', 'no')
print(f"Grant vote by Hani in block with hash: {vote_hash_3}")

Block #47 created by Charlie using PoS
Grant vote by Bob in block with hash: 72cf9390aa394399c12dfc05168d21cbc73a592f285d87c32d698f38e68554a2
Block #48 created by Charlie using PoS
Grant vote by Charlie in block with hash: f0a45e1b7a8cbf786a07254042b207a8a67e73d7e0465ffcc41562cb3e4e192a
Block #49 created by Charlie using PoS
Grant vote by Hani in block with hash: 64287dd04d91cac6a9ee5525181caf20f062903fc867f0047c9d5eb3a8d18d34


In [92]:
# Eksekusi grant setelah voting
try:
    execute_grant_hash = grant_daos.execute_grant(grant_id)
    print(f"Grant executed in block with hash: {execute_grant_hash}")
except ValueError as e:
    print(f"Error executing grant: {e}")
    # Check the votes to see if the grant was approved
    grant = grant_daos.grants.get(grant_id)
    if grant:
        print(f"Grant votes: {grant['votes']}")

Error executing grant: Grant belum disetujui atau sudah dieksekusi
Grant votes: {'yes': 1, 'no': 2}


In [93]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(grant_daos.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1721140664,
        "data": "Genesis Block",
        "hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae"
    },
    {
        "index": 1,
        "previous_hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae",
        "timestamp": 1721140664,
        "data": "Block 1 data",
        "hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6"
    },
    {
        "index": 2,
        "previous_hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6",
        "timestamp": 1721140664,
        "data": "Block 2 data",
        "hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15"
    },
    {
        "index": 3,
        "previous_hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15",
        "timestamp": 1721140664,
        "data": "Block 3 data",
        "hash": "d004e80e3

### Demo DAO untuk MediaDAOs

In [94]:
class MediaDAOs(MyBlockchain):
    def __init__(self,pos):
        super().__init__(pos)
        self.proposals = {}

    def create_proposal(self, proposal_id, proposer, description):
        self.proposals[proposal_id] = {
            'proposer': proposer,
            'description': description,
            'votes': {'yes': 0, 'no': 0},
            'executed': False
        }
        data = json.dumps({
            "type": "create_proposal",
            "proposal_id": proposal_id,
            "proposer": proposer,
            "description": description
        })
        self.add_block(data)
        return self.get_latest_block().hash

    def vote_proposal(self, proposal_id, stakeholder, vote):
        if proposal_id in self.proposals:
            if vote not in ['yes', 'no']:
                raise ValueError("Vote must be 'yes' or 'no'")
            self.proposals[proposal_id]['votes'][vote] += 1
            data = json.dumps({
                "type": "vote_proposal",
                "proposal_id": proposal_id,
                "stakeholder": stakeholder,
                "vote": vote
            })
            self.add_block(data)
            return self.get_latest_block().hash
        else:
            raise ValueError("Proposal ID not found")

    def execute_proposal(self, proposal_id):
        if proposal_id in self.proposals:
            proposal = self.proposals[proposal_id]
            if proposal['votes']['yes'] > proposal['votes']['no'] and not proposal['executed']:
                # Logic to reward or execute the proposal
                proposal['executed'] = True
                data = json.dumps({
                    "type": "execute_proposal",
                    "proposal_id": proposal_id
                })
                self.add_block(data)
                return self.get_latest_block().hash
            else:
                raise ValueError("Proposal not approved or already executed")
        else:
            raise ValueError("Proposal ID not found")

In [95]:
# Demonstrasi Penggunaan
pos = ProofOfStake()
media_daos = MediaDAOs(pos)

pos.add_stakeholder('Alice', 10)
pos.add_stakeholder('Bob', 20)
pos.add_stakeholder('Charlie', 30)
pos.add_stakeholder('Hani', 25)

In [96]:
# Menambahkan blok biasa
media_daos.add_block('Block 1 Data')
media_daos.add_block('Block 2 Data')

Block #50 created by Hani using PoS
Block #51 created by Charlie using PoS


In [97]:
# Membuat proposal media
proposal_id = "proposal1"
proposer = "Alice"
description = "Create a documentary on blockchain technology"
create_proposal_hash = media_daos.create_proposal(proposal_id, proposer, description)
print(f"Media proposal created in block with hash: {create_proposal_hash}")

Block #52 created by Bob using PoS
Media proposal created in block with hash: d9a7db4ce98093300c5b506493297a4c9bffa714c7d88be3d47045dab96d0e4b


In [98]:
# Stakeholder voting
vote_hash_1 = media_daos.vote_proposal(proposal_id, 'Bob', 'yes')
print(f"Proposal vote by Bob in block with hash: {vote_hash_1}")

vote_hash_2 = media_daos.vote_proposal(proposal_id, 'Charlie', 'yes')
print(f"Proposal vote by Charlie in block with hash: {vote_hash_2}")

vote_hash_3 = media_daos.vote_proposal(proposal_id, 'Hani', 'yes')
print(f"Proposal vote by Hani in block with hash: {vote_hash_3}")

Block #53 created by Alice using PoS
Proposal vote by Bob in block with hash: a8cf61b98910a47c23f4ea18b700dc8bf8a9c85344c909f3d88039daf5325212
Block #54 created by Alice using PoS
Proposal vote by Charlie in block with hash: f1379a2c3bd0ca067547f1678bc7afaa8162f51de50e97f234b72617e1ee9753
Block #55 created by Hani using PoS
Proposal vote by Hani in block with hash: 19f1425e2b9b7456a402f430a2171e0c2972e80d661b76bf8422d1ef380a168e


In [99]:
# Eksekusi proposal setelah voting
try:
    execute_proposal_hash = media_daos.execute_proposal(proposal_id)
    print(f"Proposal executed in block with hash: {execute_proposal_hash}")
except ValueError as e:
    print(f"Error executing proposal: {e}")
    # Check the votes to see if the proposal was approved
    proposal = media_daos.proposals.get(proposal_id)
    if proposal:
        print(f"Grant votes: {proposal['votes']}")

Block #56 created by Hani using PoS
Proposal executed in block with hash: 8de0dc0a45d53d6287524eaaa9a46ae97b72d5b0e3f62bc8e2608db8ad293d87


In [100]:
# Mencetak isi blockchain dalam format JSON
print("Blockchain content in JSON:")
print(media_daos.to_json())

Blockchain content in JSON:
[
    {
        "index": 0,
        "previous_hash": "0",
        "timestamp": 1721140664,
        "data": "Genesis Block",
        "hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae"
    },
    {
        "index": 1,
        "previous_hash": "88fba1e4805c082f7eef888527b257618c57d84cb96a55d54b54978e0c8dceae",
        "timestamp": 1721140664,
        "data": "Block 1 data",
        "hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6"
    },
    {
        "index": 2,
        "previous_hash": "dea01905eb123f2ff99d749a7884ca87ea640538ad08d826eda7b73b3401c8d6",
        "timestamp": 1721140664,
        "data": "Block 2 data",
        "hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15"
    },
    {
        "index": 3,
        "previous_hash": "e842d6276f2069f9326d753fb07756d7f757a6ab1f992f0f10d202414ab3bc15",
        "timestamp": 1721140664,
        "data": "Block 3 data",
        "hash": "d004e80e3

## DEFI

### Demo DeFi Yield Farming Platform

In [101]:
# DeFi Yield Farming Platform
class DeFiYieldFarming:
    def __init__(self, staking_token: ERC20, reward_token: ERC20, reward_rate: float):
        self.staking_token = staking_token
        self.reward_token = reward_token
        self.reward_rate = reward_rate
        self.stakes = {}
        self.rewards = {}
        self.last_update_time = {}

    def _update_rewards(self, user):
        if user in self.stakes:
            time_now = time.time()
            staked_time = time_now - self.last_update_time.get(user, time_now)
            self.rewards[user] = self.rewards.get(user, 0.0) + staked_time * self.stakes[user] * self.reward_rate
            self.last_update_time[user] = time_now

    def stake(self, user, amount):
        self._update_rewards(user)
        if self.staking_token.transfer(user, self, amount):
            self.stakes[user] = self.stakes.get(user, 0) + amount
            return True
        return False

    def unstake(self, user, amount):
        self._update_rewards(user)
        if self.stakes.get(user, 0) >= amount:
            self.stakes[user] -= amount
            if self.staking_token.transfer(self, user, amount):
                return True
            else:
                self.stakes[user] += amount  # Rollback on failure
        return False

    def claim_rewards(self, user):
        self._update_rewards(user)
        reward = self.rewards.get(user, 0.0)
        if reward > 0:
            self.rewards[user] = 0.0
            self.reward_token.balances[self] -= reward
            self.reward_token.balances[user] += reward
            return reward
        return 0.0

In [112]:
# Demo DeFi Yield Farming Platform
pos_protocol = ProofOfStake()
blockchain = MyBlockchain(pos_protocol)

reward_token = ERC20()
yield_farming_platform = DeFiYieldFarming(staking_token=blockchain.fungible_token_contract, reward_token=reward_token, reward_rate=0.1)
reward_token.mint(yield_farming_platform, 1000)

pos_protocol.add_stakeholder("alice", 100)

# Mint tokens for Alice so she has enough balance to stake
blockchain.fungible_token_contract.mint("alice", 100)  # Add this line

print("\nAlice stakes 100 fungible tokens into Yield Farming")
success = yield_farming_platform.stake("alice", 100) # Check if stake was successful
print(f"Staking successful: {success}") # Print the result
if success: # Only print balance if staking was successful
    print(f"Alice's staked balance: {yield_farming_platform.stakes['alice']}")

# Simulate time passing for rewards
import time as t
t.sleep(2)  # Simulate 2 seconds passing

print("\nAlice claims rewards from Yield Farming")
rewards = yield_farming_platform.claim_rewards("alice")
print(f"Alice's rewards: {rewards} fungible tokens")


Alice stakes 100 fungible tokens into Yield Farming
Staking successful: False

Alice claims rewards from Yield Farming
Alice's rewards: 0.0 fungible tokens


### Demo DeFi Decentralized Exchange (DEX) Platform

In [125]:
class DeFiDEX:
    def __init__(self, token1: ERC20, token2: ERC20):
        self.token1 = token1
        self.token2 = token2
        self.liquidity_pool1 = {}
        self.liquidity_pool2 = {}
        self.total_liquidity_token1 = 0
        self.total_liquidity_token2 = 0

    def add_liquidity(self, user, amount1, amount2):
        if self.token1.balance_of(user) < amount1 or self.token2.balance_of(user) < amount2:
            raise ValueError("User does not have enough tokens")
        self.token1.transfer(user, "dex_contract", amount1)
        self.token2.transfer(user, "dex_contract", amount2)
        self.liquidity_pool1[user] = self.liquidity_pool1.get(user, 0) + amount1
        self.liquidity_pool2[user] = self.liquidity_pool2.get(user, 0) + amount2
        self.total_liquidity_token1 += amount1
        self.total_liquidity_token2 += amount2
        return True

    def remove_liquidity(self, user, amount1, amount2):
        if self.liquidity_pool1.get(user, 0) >= amount1 and self.liquidity_pool2.get(user, 0) >= amount2:
            self.liquidity_pool1[user] -= amount1
            self.liquidity_pool2[user] -= amount2
            self.total_liquidity_token1 -= amount1
            self.total_liquidity_token2 -= amount2
            if self.token1.transfer("dex_contract", user, amount1) and self.token2.transfer("dex_contract", user, amount2):
                return True
            else:
                self.liquidity_pool1[user] += amount1  # Rollback on failure
                self.liquidity_pool2[user] += amount2  # Rollback on failure
                self.total_liquidity_token1 += amount1
                self.total_liquidity_token2 += amount2
        return False

    def swap(self, user, from_token, amount):
        if from_token == self.token1:
            if self.total_liquidity_token1 == 0 or self.total_liquidity_token2 == 0:
                raise ValueError("Liquidity pool is empty")
            required_amount2 = amount * self.total_liquidity_token2 / self.total_liquidity_token1
            if self.token1.transfer(user, "dex_contract", amount) and self.token2.transfer("dex_contract", user, required_amount2):
                self.total_liquidity_token1 += amount
                self.total_liquidity_token2 -= required_amount2
                return True
        elif from_token == self.token2:
            if self.total_liquidity_token1 == 0 or self.total_liquidity_token2 == 0:
                raise ValueError("Liquidity pool is empty")
            required_amount1 = amount * self.total_liquidity_token1 / self.total_liquidity_token2
            if self.token2.transfer(user, "dex_contract", amount) and self.token1.transfer("dex_contract", user, required_amount1):
                self.total_liquidity_token2 += amount
                self.total_liquidity_token1 -= required_amount1
                return True
        return False

In [126]:
# Create instances of ERC20 tokens
tokenA = ERC20()
tokenB = ERC20()

# Mint tokens for user 'alice' and 'bob'
tokenA.mint("alice", 1000)
tokenB.mint("alice", 1000)
tokenA.mint("bob", 500)
tokenB.mint("bob", 500)

# Create the DEX platform
dex_platform = DeFiDEX(tokenA, tokenB)

# Adding initial liquidity to the DEX platform
dex_platform.add_liquidity("alice", 100, 200)

# Check the liquidity pool balances
print(f"Liquidity Pool TokenA: {dex_platform.total_liquidity_token1}")
print(f"Liquidity Pool TokenB: {dex_platform.total_liquidity_token2}")

# Example usage
print("\n--- DeFi DEX Example ---")
dex_platform.swap("bob", tokenA, 50)
print(f"Bob's TokenB balance: {tokenB.balance_of('bob')}")
print(f"Liquidity Pool TokenA: {dex_platform.total_liquidity_token1}")
print(f"Liquidity Pool TokenB: {dex_platform.total_liquidity_token2}")

Liquidity Pool TokenA: 100
Liquidity Pool TokenB: 200

--- DeFi DEX Example ---
Bob's TokenB balance: 500
Liquidity Pool TokenA: 100
Liquidity Pool TokenB: 200


### Demo DeFi Staking Platform

In [130]:
class DeFiStaking:
    def __init__(self, staking_token: ERC20, reward_token: ERC20, reward_rate: float):
        self.staking_token = staking_token
        self.reward_token = reward_token
        self.reward_rate = reward_rate
        self.stakes = {}
        self.reward_debt = {}
        self.last_update_time = {}
        self.total_staked = 0

    def stake(self, user, amount):
        if self.staking_token.balance_of(user) < amount:
            raise ValueError("Insufficient staking token balance")
        self.staking_token.transfer(user, "staking_contract", amount)
        self.total_staked += amount
        if user in self.stakes:
            self.stakes[user] += amount
        else:
            self.stakes[user] = amount
        self.reward_debt[user] = self.reward_debt.get(user, 0) + self.pending_rewards(user)
        self.last_update_time[user] = time.time()

    def pending_rewards(self, user):
        if user not in self.stakes:
            return 0
        time_diff = time.time() - self.last_update_time.get(user, time.time())
        return self.stakes[user] * self.reward_rate * time_diff

    def claim_rewards(self, user):
        rewards = self.pending_rewards(user)
        if rewards > self.reward_token.balance_of("staking_contract"):
            raise ValueError("Insufficient reward token balance in contract")
        self.reward_debt[user] = 0
        self.last_update_time[user] = time.time()
        self.reward_token.transfer("staking_contract", user, rewards)
        return rewards

In [131]:
import time as t

# Create instances of ERC20 tokens
tokenA = ERC20()
reward_token = ERC20()

# Mint tokens for user 'alice' and 'bob'
tokenA.mint("alice", 1000)
reward_token.mint("staking_contract", 1000)

# Create the DeFi Staking platform
staking_platform = DeFiStaking(staking_token=tokenA, reward_token=reward_token, reward_rate=0.1)

print("\nAlice stakes 100 TokenA into Staking Platform")
staking_platform.stake("alice", 100)
print(f"Alice's staked balance: {staking_platform.stakes['alice']}")

# Simulate time passing for rewards
t.sleep(2)  # Simulate 2 seconds passing

print("\nAlice claims rewards from Staking Platform")
staking_rewards = staking_platform.claim_rewards("alice")
print(f"Alice's staking rewards: {staking_rewards} fungible tokens")


Alice stakes 100 TokenA into Staking Platform
Alice's staked balance: 100

Alice claims rewards from Staking Platform
Alice's staking rewards: 20.03138542175293 fungible tokens
